# Week 13: XPath part 1

Over this week and next week we'll be going over XPath, but also discovering more about how to parse over multiple files and do more advanced stuff with lists and nested accumulator patterns.

## What is XML?

If you truly have zero knowledge of XML, I invite you to start with the a good skim of the [Wikipedia page](https://en.wikipedia.org/wiki/XML) on the subject. Don't pour over it, but it'll provide some important background vocabulary and context.  Anyhow, XML is ruleset for marking up documents in specific ways, and has been extended to a method of storing data in a very structured way.  Instead of having a row/column structure like a CSV file, you can have nested and thus much more complex data storage this way.

Much of library metadata is stored in XML marked up documents, and that's the focus of the Metadata in Theory and Practice class offered at the iSchool.  Meanwhile, HTML is another markup language that works very similarly to XML.  Unless the HTML is severely malformed,techniques to extract data out of XML will also be useful for extracting data out of web pages.

## What is XPath?

XPath (https://en.wikipedia.org/wiki/XPath) is a query lanaguage (a la SQL, kind of) used to describe both locations for items and data extraction for XML documents/data.  This means that you can use it to both locate a specific element within an XML document but it also includes functions to pull out desired values.  Much of the time that's the text of that element, but sometimes you'll want other stuff.

XPath is a system that is platform and tool independent, and thus you can actually find tools for it in the Oxygen XML editor, and there are a few other resources.  There are many Python tools that utilize XPath and have functions for applying XPath queries, but we're going to explore one of those.  

## Installing lxml

This will be our first instance into a third party module that you'll need to install.  lxml is a module available from PyPi, which means you can use pip to install it.  Please follow these directions:

1. Open up your terminal or command prompt (this is the same as you did when you were testing your anaconda installation at the beginning of the semester.
2. Type in `pip install lxml` and press enter.  You should not get an error.
3. It should begin a downloading process and not end in a "failed" statement.
4. Once you're back to the normal command prompt, type in `python` to open up IDLE.  Again, exactly how you did when testing out Anaconda.
5. Type in `from lxml import etree` and press enter.  You should not see anything returned.  Let me know if you get an error an what that error is.
6. Download a copy of the base script from the assignment and attempt to run it inside of PyCharm.  If you passed step 5 without an error, it should run without a problem.  
    * Remember that this script requires that you have the files in a place the script can find them.  By default, it is expecting them to be in a folder a the same level of the script itself.  The script will run and execute with no data if it can't find the files, so a lack of error won't help you.  The start of the program has a statement to print out the file names it finds, so you should see something appear in your ouput.  Printing of an empty list means that it didn't find anything.

# Designing around juicy pieces of code

As you have discovered with other programs, there's a lot of data preparation and handling that go into our code before we get into the interesting pieces with XPath.  Just beacuse we're going to be using this XPath function to get out the data we want doesn't mean that the work is done for us.  That would be the case if we only needed one data point from one file.  But when we want to get multiple data points from multiple files, we need to have structures in place to collect that data and then export it to our desired structure.

So it very much is the case that you'll have one really juicy piece of code sitting in the middle that's doing the substantive data work, but you still have a ton of code before and after to prepare and output the data.  What we need to do is explore larger design questions.

You almost never know exactly how you're going to get from point A to point B in a program, but you know what point A is and what point B should be.  You may have to work a bit at the beginning, then a bit at the end, then fill things in the middle as you go.  You've see me do this during lecture, where we fill in some of our known structure and output at the start so we can focus on the meaty bits in the center.

## So what do we know about this homework assignment?

**Starting point:** You'll have 5 XML files that contain TEI-ed entries for a Digital Humanities conference.  These contain descriptive and administrative information about the entries.

**Ending point:** You need to create 3 CSV documents with data about each of these files.  2 hour students will be pulling in 2 columns of data in each file and 4 credit hour students will be pulling out 4 columns of data in each.

# Starting with multiple files

We're used to reading in one file at a time, and naming them explicitly within our programs.  Instead of this.